## 임상시험 데이터: DB Specification Sheet 보기
### 목적
DB Specsheet는 CRF 화면 구성뿐만 아니라 모든 데이터셋(sas7bdat/xport)의 컬럼 사양을 포함한다. DB Specsheet를 바탕으로 각 데이터셋의 컬럼 사양을 추출하려 한다.

### 준비
임상시험 데이터: [cafe.naver.com/dmisimportant/104](cafe.naver.com/dmisimportant/104)

### Libraries

In [132]:
import os
import pandas as pd

### Variables
Specsheet, 데이터셋 경로 설정

In [133]:
ornament="-"*10
ext=(".sas7bdat",".xport")
path_spec="C:/code/CUBEDEMO2017/spec.xlsx"
path_set="C:/code/CUBEDEMO2017/SASSET/"

#### 데이터셋: 로드
데이터셋 로드 및 이상 여부 확인

In [134]:
sasobj=[obj for obj in os.scandir(path_set) if any(map(obj.path.lower().__contains__,ext)) and obj.is_file()]
sasbad=[obj for obj in sasobj if obj.stat().st_size<3]
if len(sasbad)>1:raise Exception("exotic file exists")
print(ornament,"number of loaded sas7bdat:",len(sasobj))

def _decode(filepath):
    data=pd.read_sas(filepath)
    nas=data.notna().value_counts().sum()
    bytecol=data.select_dtypes("object").columns
    data[bytecol]=data[bytecol].apply(lambda q:q.str.decode("utf-8"))
    if nas==data.notna().value_counts().sum():
        return data
    else:
        print(ornament,"error:",filepath)
        return None

data={os.path.splitext(obj.name)[0].upper():_decode(obj.path) for obj in sasobj}
print(ornament,"domain:",os.linesep,data.keys(),os.linesep,len(data),"domains")

---------- number of loaded sas7bdat: 27
---------- domain: 
 dict_keys(['AE', 'AY', 'CM', 'CT', 'CY', 'DA', 'DM', 'DS', 'DY', 'EF', 'EG', 'EN', 'ES', 'IE', 'IP', 'LB', 'LC', 'LY', 'MH', 'MY', 'PD', 'PG', 'RN', 'SN', 'SU', 'SV', 'VS']) 
 27 domains


#### 스펙시트: 로드
스펙시트 로드 및 형태 확인
- 100% 0 non-null 컬럼 제거

In [135]:
specsheet=pd.read_excel(path_spec)
print(specsheet.info())
specsheet=specsheet.dropna(how="all",axis=1)
specsheet.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291 entries, 0 to 290
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   DOMAIN           291 non-null    object 
 1   PGNM             291 non-null    object 
 2   PGNO             291 non-null    int64  
 3   PAGE_LABEL       291 non-null    object 
 4   CRF_LABEL        291 non-null    object 
 5   VISIT            291 non-null    object 
 6   ITEMID           291 non-null    object 
 7   ITEM_SEQ         291 non-null    int64  
 8   ITEM_LABEL       291 non-null    object 
 9   CODE             107 non-null    object 
 10  LAYOUT           291 non-null    object 
 11  KEY              113 non-null    float64
 12  TYPE_LENGTH      291 non-null    object 
 13  VIEW_TYPE        291 non-null    object 
 14  DERIVED          0 non-null      float64
 15  DERIVED_EXPLAIN  0 non-null      float64
 16  COMMENT          0 non-null      float64
 17  ISSUE           

Index(['DOMAIN', 'PGNM', 'PGNO', 'PAGE_LABEL', 'CRF_LABEL', 'VISIT', 'ITEMID',
       'ITEM_SEQ', 'ITEM_LABEL', 'CODE', 'LAYOUT', 'KEY', 'TYPE_LENGTH',
       'VIEW_TYPE'],
      dtype='object')

#### 스펙시트: 컬럼 정의 대상 파악
스펙시트는 CRScube 사양이라고 가정
- 스펙시트의 모든 컬럼이 CDMS/CRF/Validation에 사용됨
- 스펙시트 컬럼명과 데이터셋 컬럼명이 다를뿐 데이터셋의 모든 컬럼은 스펙시트 내 정의됨

In [136]:
dataset_col=[]
[dataset_col.extend(q) for q in [w.columns for w in data.values()]]
dataset_col_wo_spec=[q for q in dataset_col if not q in specsheet.ITEMID.unique()]
print(ornament,"dataset column without spec.:",len(dataset_col_wo_spec),dataset_col_wo_spec)

---------- dataset column without spec.: 0 []


#### 스펙시트와 데이터셋의 대응 컬럼 파악
데이터셋은 [DOMAIN..VARNAME]로서 스펙시트에 대응
- 이를 바탕으로 스펙시트의 viewport 또는 attribute를 내고자 함

In [137]:
mh=data["MH"]
specsheet[specsheet.DOMAIN=="MH"].head(5)

,DOMAIN,PGNM,PGNO,PAGE_LABEL,CRF_LABEL,VISIT,ITEMID,ITEM_SEQ,ITEM_LABEL,CODE,LAYOUT,KEY,TYPE_LENGTH,VIEW_TYPE
21,MH,MH,6,Medical History,Medical History,1,SUBJID,1,Screening Number,NaN,SYSDEFINED,1.0,C8,nvarchar2(8)
22,MH,MH,6,Medical History,Medical History,1,SEQ,2,Seq,NaN,SYSDEFINED,1.0,N,num
23,MH,MH,6,Medical History,Medical History,1,MHTERM,3,Medical history term,NaN,MEDCOD,NaN,C255,nvarchar2(255)
24,MH,MH,6,Medical History,Medical History,1,MHONGO,4,Ongoing,1:Yes|2:No,RADIO,NaN,N2,num
25,MH,MH,6,Medical History,Medical History,1,MHENDTC,5,End date,NaN,DATE,NaN,YYYY-UK-UK,nvarchar2(10)


- DOMAIN, VARNAME을 인덱스로 데이터셋 스펙을 가져오는 클래스를 구성
    - 이렇게 하는 것이 거시적이면서 레코드의 조건을 감안하는 최선으로 생각됨
    - 팬더스의 컬럼 upcasting 문제도 해결할 수 있음

In [138]:
class _spec:
	def __init__(self,specsheet):
		spec=specsheet.set_index(["DOMAIN","ITEMID"])
		spec_dupe=spec.index.duplicated()
		if any(spec_dupe):
			return spec[spec_dupe]
		spec.index.names=["DOMAIN","VARNAME"]
		self.spec=spec
		[setattr(self,ix,spec.loc[ix]) for ix in spec.index.unique(level="DOMAIN")] # DOMAIN

		for ix in spec.index:
			_spec=list(zip(spec,spec.loc[ix]))
			setattr(self,f"{ix[0]}_{ix[1]}",_spec) # DOMAIN.VARNAME
			[self.__setattr__(f"{ix[0]}_{ix[1]}_{q[0]}",q[1]) for q in _spec] # DOMAIN.VARNAME.COLUMN 

spec=_spec(specsheet)
dir(spec)[:10]

['AE',
 'AE_AEACN',
 'AE_AEACNOTH',
 'AE_AEACNOTH_CODE',
 'AE_AEACNOTH_CRF_LABEL',
 'AE_AEACNOTH_ITEM_LABEL',
 'AE_AEACNOTH_ITEM_SEQ',
 'AE_AEACNOTH_KEY',
 'AE_AEACNOTH_LAYOUT',
 'AE_AEACNOTH_PAGE_LABEL']

- spec에서 DOMAIN / VARNAME 별 레코드 스펙을 쿼리 가능

In [139]:
print(ornament,spec.AE_AEACN_ITEM_LABEL)
print(ornament,spec.AE_AEACN_KEY)
spec.AE.sample(5)

---------- Action taken with study treatment
---------- nan


,PGNM,PGNO,PAGE_LABEL,CRF_LABEL,VISIT,ITEM_SEQ,ITEM_LABEL,CODE,LAYOUT,KEY,TYPE_LENGTH,VIEW_TYPE
VARNAME,,,,,,,,,,,,
INV_PT,AE,23,Adverse Event,Adverse Event,"3003,5003",34,MedDRA_INV_Preferred Term,NaN,SYSDEFINED,1.0,C100,nvarchar2(100)
INV_HLGT,AE,23,Adverse Event,Adverse Event,"3003,5003",17,MedDRA_INV_High Level Group Term,NaN,SYSDEFINED,1.0,C100,nvarchar2(100)
INV_HLGTCD,AE,23,Adverse Event,Adverse Event,"3003,5003",37,MedDRA_INV_High Level Group Term Code,NaN,SYSDEFINED,1.0,N,num
AEOUT,AE,23,Adverse Event,Adverse Event,"3003,5003",7,Outcome,1:Fatal|2:Not recovered/Not resolved|3:Recover...,DROPDOWN,NaN,N2,num
PTCD,AE,23,Adverse Event,Adverse Event,"3003,5003",26,MedDRA_DM_Preferred Term Code,NaN,SYSDEFINED,1.0,N,num


- TYPE_LENGTH(CRF 상 edit check 용), VIEW_TYPE(DB 상 type)로 기술적 무결성 확인이 가능
    - 예컨대 SUBJID는 C8(character 8)이어야 함

In [140]:
def get_ectl(type_length):
    typechar=type_length[0]
    if typechar=="C":
        return str,int(type_length[1:])
    elif typechar=="N":
        lenchar=type_length[1:]
        if "." in lenchar:
            deci=lenchar.index(".")
            x0=lenchar[:deci]
            x1=lenchar[deci+1:]
            return float,sum(map(int,(x0,x1)))+1
        else:
            return float,int(lenchar)
    else:
        raise NotImplementedError("")

def ec(data,ect):
    data=data.to_frame()
    data["_TYPE"]=[isinstance(q,ect[0]) for q in data.iloc[:,0]]
    if all(data._TYPE):
        data["_LEN"]=data.iloc[:,0].str.len()==ect[1]
        if all(data._LEN):
            return True
    return data[~(data._TYPE+data._LEN)]

In [141]:
get_ectl(spec.AE_SUBJID_TYPE_LENGTH)

(str, 8)

In [142]:
ec(mh.SUBJID,get_ectl(spec.AE_SUBJID_TYPE_LENGTH))

True

- mh.SUBJID는 문제가 없음

### 후기
- Specsheet에서 KEY가 1이 아닐 때 빈 값이 허용됨
    - Optional, conditional인 CRF 항목
    - longitudinal이거나 visit site, visit time, sequence에 따라 하나의 테이블에도 long / wide로 다루어야 하는 부분이 혼재
        - 결과적으로 per-row로 다루어야 함
    - DOMAIN에 따라 categorise 하는 것은 의미가 없음
        - Per-row spec을 다루어야 함
- 프로토콜 및 당국 규제에 의거하되, CRF 항목은 최소화하고, production이면 end user에게 모든 항목은 가능한 채우도록 함